In [42]:
#Import libraries
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score as ss
import pandas as pd 
import numpy as np
import plotly.express as px
import itertools as it 
import folium
from folium.plugins import GroupedLayerControl
import webbrowser

In [43]:
#Original dataframe
original_df = pd.read_csv('Street Data_BaDinh.csv')
#Drop duplicates
new_df = original_df.drop_duplicates(subset=['Longitude','Latitude'])
new_df.head(10)

,Index,District,Ward,Street Name 1,Street name 2,Street Name 3,Street Name 4,Longitude,Latitude,Traffic Level
0,10101,Ba Đình,Phúc Xá,An Xá,Nghĩa Dũng,NaN,NaN,21.046224,105.849289,1
1,10102,Ba Đình,Phúc Xá,An Xá,Tân Mỹ,NaN,NaN,21.042412,105.852127,1
2,10103,Ba Đình,Phúc Xá,An Xá,Phúc Xá,NaN,NaN,21.046067,105.849022,1
3,11901,Ba Đình,Điện Biên,Bà Huyện Thanh Quan,Chùa Một Cột,NaN,NaN,21.034974,105.835874,1
4,11902,Ba Đình,Điện Biên,Bà Huyện Thanh Quan,Lê Hồng Phong,NaN,NaN,21.033849,105.835607,2
5,11903,Ba Đình,Điện Biên,Bắc Sơn,Hoàng Diệu,NaN,NaN,21.035962,105.839265,1
6,11904,Ba Đình,Điện Biên,Bắc Sơn,Tôn Thất Đàm,NaN,NaN,21.035980,105.838074,1
7,11905,Ba Đình,Điện Biên,Bắc Sơn,Độc Lập,NaN,NaN,21.036582,105.836578,1
8,11601,Ba Đình,Ngọc Hà,Bắc Sơn,Ngọc Hà,NaN,NaN,21.037553,105.831080,1
9,10701,Ba Đình,Cống Vị,Bưởi,Đội Cấn,NaN,NaN,21.037688,105.807348,3


In [44]:
#Count Missing Value
print(new_df.isnull().sum(), "\n")
#Check Datatype
print(new_df.dtypes)

Index              0
District           0
Ward               0
Street Name 1      0
Street name 2      0
Street Name 3    165
Street Name 4    194
Longitude          0
Latitude           0
Traffic Level      0
dtype: int64 

Index              int64
District          object
Ward              object
Street Name 1     object
Street name 2     object
Street Name 3     object
Street Name 4     object
Longitude        float64
Latitude         float64
Traffic Level      int64
dtype: object


In [45]:
fig = px.bar(new_df, x="Ward", color=new_df["Traffic Level"].astype(str), color_discrete_map={"3": "red","1": "green","2": "yellow"})
fig.show()

In [46]:
#Cordinates and Traffic Level
main = new_df.iloc[:,[7,8,9]]
main.head(10)

,Longitude,Latitude,Traffic Level
0,21.046224,105.849289,1
1,21.042412,105.852127,1
2,21.046067,105.849022,1
3,21.034974,105.835874,1
4,21.033849,105.835607,2
5,21.035962,105.839265,1
6,21.035980,105.838074,1
7,21.036582,105.836578,1
8,21.037553,105.831080,1
9,21.037688,105.807348,3


In [47]:
#Cordinates for Traffic Level 1 
TF_1 = main[main['Traffic Level'] == 1].iloc[:,[0,1]]
TF_1.head(10)

,Longitude,Latitude
0,21.046224,105.849289
1,21.042412,105.852127
2,21.046067,105.849022
3,21.034974,105.835874
5,21.035962,105.839265
6,21.035980,105.838074
7,21.036582,105.836578
8,21.037553,105.831080
20,21.042534,105.841838
21,21.043551,105.842065


In [48]:
#Cordinates for Traffic Level 2
TF_2 = main[main['Traffic Level'] == 2].iloc[:,[0,1]]
TF_2.head(10)

,Longitude,Latitude
4,21.033849,105.835607
10,21.036802,105.806949
14,21.030388,105.838591
15,21.029578,105.841557
16,21.030178,105.836997
18,21.028600,105.805231
19,21.028430,105.805509
25,21.034975,105.836843
26,21.033652,105.836641
33,21.040651,105.842084


In [49]:
#Cordinates for Traffic Level 3
TF_3 = main[main['Traffic Level'] == 3].iloc[:,[0,1]]
TF_3.head(10)

,Longitude,Latitude
9,21.037688,105.807348
11,21.034335,105.805245
12,21.029744,105.802636
13,21.047508,105.806128
27,21.031582,105.836236
28,21.030390,105.836007
38,21.045600,105.843339
44,21.032382,105.809576
48,21.030593,105.815841
49,21.023419,105.819744


#Training Model for Traffic Level 1
model1 = DBSCAN(0.005,min_samples=5).fit(cluster1)
TF_1['labels'] = model1.labels_
print((TF_1['labels']).value_counts())
score1 = ss(cluster1, TF_1['labels'])
score1

after_fig1 = px.scatter(TF_1, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_1['labels'].astype(str),
                        labels={"color" : "Cluster"},
                      ) #Default eps, min
after_fig1.update_layout(
            title={
            'text' : "Area with traffic level 1",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig1

#Training Model for Traffic Level 2
model2 = DBSCAN(0.005, min_samples=5).fit(cluster2)
TF_2['labels'] = model2.labels_
print(TF_2['labels'].value_counts())
score2 = ss(cluster2, TF_2['labels'])
score2

after_fig2 = px.scatter(TF_2,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_2['labels'].astype(str),
                        labels={"color" : "Cluster"},
                        )
after_fig2.update_layout(
            title={
            'text' : "Area with traffic level 2",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig2

#Training Model for Traffic Level 3
model3 = DBSCAN(0.005,min_samples=5).fit(cluster3) 
TF_3['labels']= model3.labels_
print(TF_3['labels'].value_counts())
score3 = ss(cluster3, TF_3['labels'])
score3

after_fig3 = px.scatter(TF_3, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_3['labels'].astype(str),
                        labels={"color" : "Cluster"})
after_fig3.update_layout(
            title={
            'text' : "Area with traffic level 3",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig3

In [50]:
#Setting Param ranges
cluster1 = TF_1.to_numpy()
cluster2 = TF_2.to_numpy()
cluster3 = TF_3.to_numpy()
epsilon1 = np.linspace(0.0003, 0.0049, num=1000).tolist()
epsilon2 = np.linspace(0.0013, 0.0073, num=1000).tolist()
epsilon3 = np.linspace(0.001, 0.0098, num=1000).tolist()
min_samples = [3,4,5,6,7]

In [51]:
#Function to search through Parameters
def getScore(dataset,epss,min_sampless,min_score,min_num_clus):
    if epss == epsilon1:
        cluster = cluster1
    elif epss == epsilon2:
        cluster = cluster2
    elif epss == epsilon3:
        cluster = cluster3
    loop_count = 1
    #Loop through parameters from combinations
    for i, (epsilon, min_sample) in enumerate(list(it.product(epss, min_sampless))):
        #Fit model
        model = DBSCAN(eps = epsilon, min_samples = min_sample).fit(cluster)
        #Get lables
        dataset['labels'] = model.labels_
        num_clus = len(dataset['labels'].value_counts()) 
        #Get scores
        if num_clus > 2:
            score = ss(cluster, dataset['labels'])
            if score >= min_score and num_clus > min_num_clus: 
                print("Loop number:",loop_count,"| Parameters:", epsilon,",", min_sample,"| Score for Trafic Level 1:", score,"| Number of clusters:", num_clus)
        loop_count += 1

In [52]:
getScore(TF_1,epsilon1,min_samples,min_score=0.414,min_num_clus=3)

Loop number: 2890 | Parameters: 0.0029568568568568566 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2895 | Parameters: 0.0029614614614614614 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2900 | Parameters: 0.002966066066066066 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2905 | Parameters: 0.0029706706706706704 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2910 | Parameters: 0.002975275275275275 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2915 | Parameters: 0.00297987987987988 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2920 | Parameters: 0.0029844844844844846 , 7 | Score for Trafic Level 1: 0.4148832460474392 | Number of clusters: 4
Loop number: 2925 | Parameters: 0.002989089089089089 , 7 | Score for Trafic Level 1: 0.41488324604743

In [53]:
getScore(TF_2,epsilon2,min_samples,min_score=0.461,min_num_clus=3)

Loop number: 2917 | Parameters: 0.0048015015015015015 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2922 | Parameters: 0.004807507507507507 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2927 | Parameters: 0.004813513513513513 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2932 | Parameters: 0.004819519519519519 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2937 | Parameters: 0.004825525525525526 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2942 | Parameters: 0.0048315315315315315 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4
Loop number: 2947 | Parameters: 0.004837537537537537 , 4 | Score for Trafic Level 1: 0.4610507666492643 | Number of clusters: 4


In [54]:
getScore(TF_3,epsilon3,min_samples,min_score=0.517,min_num_clus=3)

Loop number: 2726 | Parameters: 0.005800800800800801 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2731 | Parameters: 0.00580960960960961 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2736 | Parameters: 0.005818418418418418 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2741 | Parameters: 0.0058272272272272265 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2746 | Parameters: 0.0058360360360360354 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2751 | Parameters: 0.005844844844844844 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2756 | Parameters: 0.005853653653653653 , 3 | Score for Trafic Level 1: 0.5171859719938379 | Number of clusters: 4
Loop number: 2761 | Parameters: 0.005862462462462462 , 3 | Score for Trafic Level 1: 0.5171859719938379

In [66]:
#Training Model for Traffic Level 1
model1 = DBSCAN(eps=0.00296,min_samples=7).fit(cluster1)
TF_1['labels'] = model1.labels_
print((TF_1['labels']).value_counts())
score1 = ss(cluster1, TF_1['labels'])
print("Silhouette score:", score1)

 1    28
 0    15
-1    10
 2     6
Name: labels, dtype: int64
Silhouette score: 0.4148832460474392


In [56]:
#Plot for Model 1
after_fig1 = px.scatter(TF_1, 
                        x = "Latitude",
                        y = "Longitude", 
                        color=TF_1['labels'].astype(str),
                        color_discrete_map={'-1':'red', '0': '#636EFA'},
                        labels={"color" : "Cluster"},
                      ) 
after_fig1.update_layout(
            title={
            'text' : "Area with traffic level 1 with eps = 0.00296, min_samples = 7",
            'x':0.5,
            'xanchor': 'center',
        })


In [57]:
#Training Model for Traffic Level 2
model2 = DBSCAN(0.00481, min_samples=4).fit(cluster2)
TF_2['labels'] = model2.labels_
print(TF_2['labels'].value_counts())
score2 = ss(cluster2, TF_2['labels'])
print("Silhouette score:", score2)

 0    62
 1    14
-1     8
 2     7
Name: labels, dtype: int64
Silhouette score: 0.4610507666492643


In [58]:
#Plot for Model 2
after_fig2 = px.scatter(TF_2,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_2['labels'].astype(str),
                        labels={"color" : "Cluster"},
                        )
after_fig2.update_layout(
            title={
            'text' : "Area with traffic level 2 with eps = 0.00481, min_samples = 4",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig2

In [59]:
#Training Model for Traffic Level 3
model3 = DBSCAN(0.0058,min_samples=3).fit(cluster3) 
TF_3['labels']= model3.labels_
print(TF_3['labels'].value_counts())
score3 = ss(cluster3, TF_3['labels'])
print("Silhouette score:", score3)

 0    28
 1    10
 2     5
-1     4
Name: labels, dtype: int64
Silhouette score: 0.5171859719938379


In [60]:
#Plot for Model 3
after_fig3 = px.scatter(TF_3,
                        x = "Latitude" , 
                        y = "Longitude", 
                        color=TF_3['labels'].astype(str),
                        labels={"color" : "Cluster"},
                        )
after_fig3.update_layout(
            title={
            'text' : "Area with traffic level 3 with eps = 0.0058, min_samples = 3",
            'x':0.5,
            'xanchor': 'center',
        })
after_fig3

In [61]:
#Function to create map 
def createMap(dataset1, dataset2, dataset3):
    m = folium.Map(location=[21.034388, 105.831716], zoom_start=14, tiles='cartodbpositron')
    group_tf_1 = folium.FeatureGroup("Traffic Level 1").add_to(m)
    group_tf_2 = folium.FeatureGroup("Traffic Level 2").add_to(m)
    group_tf_3 = folium.FeatureGroup("Traffic Level 3").add_to(m)
    labs1 = dataset1['labels'].to_numpy()
    labs2 = dataset2['labels'].to_numpy()
    labs3 = dataset3['labels'].to_numpy()
    long_lat1 = dataset1.iloc[:,[0,1]].to_numpy()
    long_lat2 = dataset2.iloc[:,[0,1]].to_numpy()
    long_lat3 = dataset3.iloc[:,[0,1]].to_numpy()
    for i in range (0, len(labs1)):
        if labs1[i] == 0:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 1:
            folium.CircleMarker(long_lat1[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == 2: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_1)
        if labs1[i] == -1: 
            folium.CircleMarker(long_lat1[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_1)
    for i in range (0, len(labs2)):
        if labs2[i] == 0:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 1:
            folium.CircleMarker(long_lat2[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == 2: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_2)
        if labs2[i] == -1: 
            folium.CircleMarker(long_lat2[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_2)
    for i in range (0, len(labs3)):
        if labs3[i] == 0:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#636EFA",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 1:
            folium.CircleMarker(long_lat3[i], radius= 1, color="#00CC96",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == 2: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#AB63FA",  fill_opacity=0.9).add_to(group_tf_3)
        if labs3[i] == -1: 
            folium.CircleMarker(long_lat3[i], radius= 1, color="#EF553B",  fill_opacity=0.9).add_to(group_tf_3)
    GroupedLayerControl(
        groups={'Traffic Levels': [group_tf_1, group_tf_2,group_tf_3]},
        collapsed=False,
    ).add_to(m)
    folium.LayerControl(collapsed=False)
    return m


In [62]:
FinalMap=createMap(TF_1, TF_2,TF_3)

In [63]:
FinalMap.save('traffic_map.html')
webbrowser.open('traffic_map.html')

True